# Policy Realism Analysis Example

This notebook demonstrates how to perform policy realism analysis for health economic evaluation.

In [ ]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import os

# Add the scripts directory to the path
sys.path.append(os.path.join(os.pardir, 'scripts'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'models'))
sys.path.append(os.path.join(os.pardir, 'scripts', 'core'))

In [ ]:
# Define policy realism analysis functions
def assess_policy_realism(
    strategy,
    cost_effectiveness,
    budget_impact,
    implementation_feasibility,
    regulatory_pathway,
    stakeholder_acceptance,
    policy_environment,
    time_horizon=5
):
    """
    Assess policy realism for a treatment strategy.
    
    Args:
        strategy: Treatment strategy name
        cost_effectiveness: Cost-effectiveness score (1-10, higher is better)
        budget_impact: Budget impact score (1-10, higher indicates greater impact)
        implementation_feasibility: Implementation feasibility (1-10, higher is better)
        regulatory_pathway: Regulatory pathway clarity (1-10, higher is clearer)
        stakeholder_acceptance: Stakeholder acceptance (1-10, higher is better)
        policy_environment: Political/regulatory environment (1-10, higher is more favorable)
        time_horizon: Analysis time horizon
        
    Returns:
        Dictionary with policy realism assessment
    """
    # Calculate composite scores
    technical_score = (cost_effectiveness + implementation_feasibility + regulatory_pathway) / 3
    environmental_score = (stakeholder_acceptance + policy_environment) / 2
    
    # Calculate policy realism index (PRI)
    # PRI ranges from 0-100, where higher is more realistic
    pri = (technical_score * 0.6 + environmental_score * 0.4) * 10
    
    # Calculate implementation probability
    impl_prob = pri / 100  # Convert to probability
    
    # Calculate risk factors
    budget_risk = budget_impact / 10  # Higher budget impact = higher risk
    implementation_risks = []
    
    if cost_effectiveness < 4:
        implementation_risks.append("Low cost-effectiveness")
    if implementation_feasibility < 4:
        implementation_risks.append("Low implementation feasibility")
    if stakeholder_acceptance < 5:
        implementation_risks.append("Low stakeholder acceptance")
    if budget_impact > 7:
        implementation_risks.append("High budget impact")
    
    # Calculate time to implementation based on scores
    time_factor = (10 - policy_environment) * 0.5 + (10 - implementation_feasibility) * 0.3 + (10 - regulatory_pathway) * 0.2
    estimated_time = max(1, min(time_horizon, time_factor))  # Min 1 year, max time_horizon
    
    return {
        'strategy': strategy,
        'cost_effectiveness': cost_effectiveness,
        'budget_impact': budget_impact,
        'implementation_feasibility': implementation_feasibility,
        'regulatory_pathway': regulatory_pathway,
        'stakeholder_acceptance': stakeholder_acceptance,
        'policy_environment': policy_environment,
        'technical_score': technical_score,
        'environmental_score': environmental_score,
        'policy_realism_index': pri,
        'implementation_probability': impl_prob,
        'estimated_time_to_implementation': estimated_time,
        'risk_factors': implementation_risks,
        'recommendation': 'Pursue' if pri >= 70 else 'Monitor' if pri >= 40 else 'Defer'
    }

In [ ]:
# Define strategies and their policy realism parameters
strategies = ['ECT', 'IV-KA', 'PO-KA']
policy_data = [
    # ECT - Established therapy with strong evidence but moderate innovation
    {
        'cost_effectiveness': 7,
        'budget_impact': 6,  # High volume but established costs
        'implementation_feasibility': 8,  # Well-established procedure
        'regulatory_pathway': 10,  # Already approved
        'stakeholder_acceptance': 6,  # Some concerns about side effects
        'policy_environment': 7  # Well-established but evolving
    },
    # IV-KA - Novel but proven approach with higher novelty
    {
        'cost_effectiveness': 8,
        'budget_impact': 7,  # Moderate budget impact
        'implementation_feasibility': 5,  # Requires new infrastructure
        'regulatory_pathway': 6,  # Emerging regulatory guidance
        'stakeholder_acceptance': 7,  # Generally positive
        'policy_environment': 8  # Favorable for innovation
    },
    # PO-KA - Most novel approach with potential advantages
    {
        'cost_effectiveness': 8.5,
        'budget_impact': 4,  # Lower per-patient cost
        'implementation_feasibility': 7,  # Lower infrastructure requirements
        'regulatory_pathway': 4,  # Developing regulatory framework
        'stakeholder_acceptance': 8,  # Novel but promising
        'policy_environment': 8  # Very supportive of innovation
    }
]

# Calculate policy realism assessments
policy_results = []
for i, strategy in enumerate(strategies):
    result = assess_policy_realism(strategy, **policy_data[i])
    policy_results.append(result)

policy_df = pd.DataFrame(policy_results)
print(policy_df)

In [ ]:
# Visualize policy realism results
fig, ax = plt.subplots(2, 2, figsize=(16, 12))

# 1. Policy Realism Index by strategy
ax[0, 0].bar(policy_df['strategy'], policy_df['policy_realism_index'], 
             color=['blue', 'green', 'orange'], alpha=0.7)
ax[0, 0].set_ylabel('Policy Realism Index (0-100)')
ax[0, 0].set_title('Policy Realism Index by Strategy')
ax[0, 0].grid(True, alpha=0.3)

# Add value labels on bars
for i, v in enumerate(policy_df['policy_realism_index']):
    ax[0, 0].text(i, v + max(policy_df['policy_realism_index'])*0.01, 
                 f'{v:.1f}', ha='center', va='bottom')

# 2. Component scores radar chart
from math import pi
 
# Set data
cat_labels = ['Cost-Effectiveness', 'Budget Impact', 'Implementation', 
              'Regulatory', 'Stakeholder', 'Policy']
 
angles = [n / float(len(cat_labels)) * 2 * pi for n in range(len(cat_labels))]
angles += angles[:1]  # Close the circle
 
ax[0, 1] = plt.subplot(2, 2, 2, polar=True)
 
# Draw one axe per variable + add labels
plt.xticks(angles[:-1], cat_labels, color='grey', size=8)
 
# Add y-labels
ax[0, 1].set_rlabel_position(0)
plt.yticks([2,4,6,8,10], ["2","4","6","8","10"], color="grey", size=7)
plt.ylim(0, 10)
 
# Plot each strategy
for idx, row in policy_df.iterrows():
    values = [row['cost_effectiveness'], row['budget_impact'], 
              row['implementation_feasibility'], row['regulatory_pathway'], 
              row['stakeholder_acceptance'], row['policy_environment']]
    values += values[:1]  # Close the circle
    
    ax[0, 1].plot(angles, values, 'o-', linewidth=2, label=row['strategy'])
    ax[0, 1].fill(angles, values, alpha=0.25)
 
# Add a legend
plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
ax[0, 1].set_title('Policy Realism Components Radar Chart', pad=20)

# 3. Implementation probability vs time to implementation
colors = ['blue' if s == 'ECT' else 'green' if s == 'IV-KA' else 'orange' for s in policy_df['strategy']]
scatter = ax[1, 0].scatter(policy_df['implementation_probability'], 
                          policy_df['estimated_time_to_implementation'],
                          s=200, c=colors, alpha=0.7)
ax[1, 0].set_xlabel('Implementation Probability')
ax[1, 0].set_ylabel('Estimated Time to Implementation (Years)')
ax[1, 0].set_title('Implementation Probability vs Time to Implementation')
ax[1, 0].grid(True, alpha=0.3)

# Add strategy labels
for i, row in policy_df.iterrows():
    ax[1, 0].annotate(row['strategy'], 
                     (row['implementation_probability'], row['estimated_time_to_implementation']),
                     textcoords="offset points", xytext=(5,5), ha='left')

# 4. Recommendations
rec_counts = policy_df['recommendation'].value_counts()
colors = ['#2ecc71' if name == 'Pursue' else '#f39c12' if name == 'Monitor' else '#e74c3c' for name in rec_counts.index]
ax[1, 1].pie(rec_counts.values, labels=rec_counts.index, autopct='%1.1f%%', colors=colors, startangle=90)
ax[1, 1].set_title('Strategy Recommendations Distribution')

plt.tight_layout()
plt.show()

In [ ]:
# Create a policy realism matrix
plt.figure(figsize=(12, 8))

# Create a matrix for policy realism components
components = ['cost_effectiveness', 'budget_impact', 'implementation_feasibility', 
              'regulatory_pathway', 'stakeholder_acceptance', 'policy_environment']
component_names = ['Cost-Effectiveness', 'Budget Impact', 'Implementation Feasibility', 
                   'Regulatory Clarity', 'Stakeholder Acceptance', 'Policy Environment']

# Create a DataFrame with component scores
matrix_data = []
for _, row in policy_df.iterrows():
    for comp, comp_name in zip(components, component_names):
        matrix_data.append({
            'strategy': row['strategy'],
            'component': comp_name,
            'score': row[comp]
        })

matrix_df = pd.DataFrame(matrix_data)
pivot_matrix = matrix_df.pivot_table(values='score', index='strategy', columns='component')

# Create heatmap
sns.heatmap(pivot_matrix, annot=True, fmt='.1f', cbar_kws={'label': 'Score (1-10)'}, 
            center=5, cmap='RdBu_r', vmax=10, vmin=1)
plt.title('Policy Realism Component Scores by Strategy')
plt.ylabel('Strategy')
plt.xlabel('Policy Realism Component')
plt.tight_layout()
plt.show()

In [ ]:
# Perform sensitivity analysis on policy environment
env_range = np.arange(1, 11, 0.5)
sensitivity_data = []

for env_score in env_range:
    for i, strategy in enumerate(strategies):
        # Use base parameters but vary policy environment
        params = policy_data[i].copy()
        params['policy_environment'] = env_score
        
        result = assess_policy_realism(strategy, **params)
        sensitivity_data.append({
            'strategy': strategy,
            'policy_environment': env_score,
            'policy_realism_index': result['policy_realism_index'],
            'implementation_probability': result['implementation_probability'],
            'estimated_time': result['estimated_time_to_implementation']
        })

sens_df = pd.DataFrame(sensitivity_data)

# Plot sensitivity analysis
fig, ax = plt.subplots(1, 3, figsize=(18, 6))

# Policy Realism Index
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[0].plot(strat_data['policy_environment'], strat_data['policy_realism_index'], 
               label=strategy, linewidth=2)

ax[0].set_xlabel('Policy Environment Score')
ax[0].set_ylabel('Policy Realism Index')
ax[0].set_title('Policy Realism Index vs Policy Environment')
ax[0].legend()
ax[0].grid(True, alpha=0.3)

# Implementation Probability
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[1].plot(strat_data['policy_environment'], strat_data['implementation_probability'], 
               label=strategy, linewidth=2)

ax[1].set_xlabel('Policy Environment Score')
ax[1].set_ylabel('Implementation Probability')
ax[1].set_title('Implementation Probability vs Policy Environment')
ax[1].legend()
ax[1].grid(True, alpha=0.3)

# Time to Implementation
for strategy in strategies:
    strat_data = sens_df[sens_df['strategy'] == strategy]
    ax[2].plot(strat_data['policy_environment'], strat_data['estimated_time'], 
               label=strategy, linewidth=2)

ax[2].set_xlabel('Policy Environment Score')
ax[2].set_ylabel('Estimated Time to Implementation (Years)')
ax[2].set_title('Time to Implementation vs Policy Environment')
ax[2].invert_yaxis()  # Lower time is better
ax[2].legend()
ax[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Generate policy recommendations
print("POLICY REALISM ASSESSMENT")
print("="*60)

for _, row in policy_df.iterrows():
    print(f"\n{row['strategy']}:\n" + "-"*30)
    print(f"  Policy Realism Index: {row['policy_realism_index']:.1f}/100")
    print(f"  Implementation Probability: {row['implementation_probability']:.1%}")
    print(f"  Estimated Time to Implementation: {row['estimated_time_to_implementation']} years")
    print(f"  Recommendation: {row['recommendation']}")
    
    print(f"  Component Scores:")
    print(f"    - Cost-Effectiveness: {row['cost_effectiveness']}/10")
    print(f"    - Budget Impact: {row['budget_impact']}/10")
    print(f"    - Implementation Feasibility: {row['implementation_feasibility']}/10")
    print(f"    - Regulatory Pathway: {row['regulatory_pathway']}/10")
    print(f"    - Stakeholder Acceptance: {row['stakeholder_acceptance']}/10")
    print(f"    - Policy Environment: {row['policy_environment']}/10")
    
    if row['risk_factors']:
        print(f"  Risk Factors: {', '.join(row['risk_factors'])}")
    else:
        print(f"  Risk Factors: None identified")

# Overall recommendation
best_strategy = policy_df.loc[policy_df['policy_realism_index'].idxmax()]
print(f"\n\nOVERALL RECOMMENDATION: {best_strategy['strategy']} "
      f"(PRI: {best_strategy['policy_realism_index']:.1f})")

# Identify strategies with high implementation probability
high_impl_prob = policy_df[policy_df['implementation_probability'] >= 0.6]
if not high_impl_prob.empty:
    print(f"\nStrategies with High Implementation Probability (>60%):")
    for _, row in high_impl_prob.iterrows():
        print(f"  - {row['strategy']}: {row['implementation_probability']:.1%}")

# Identify strategies with shortest implementation time
shortest_time = policy_df[policy_df['estimated_time_to_implementation'] == 
                          policy_df['estimated_time_to_implementation'].min()]
print(f"\nStrategies with Shortest Implementation Time ")
for _, row in shortest_time.iterrows():
    print(f"  - {row['strategy']}: {row['estimated_time_to_implementation']} years")

## Next Steps

1. Integrate with actual policy environment data
2. Consider political and regulatory trends
3. Evaluate stakeholder engagement requirements
4. Model policy implementation pathways